#Imports

In [0]:
import pandas as pd
import numpy as np
from unidecode import unidecode
from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import seaborn as sns

import mlflow
mlflow.autolog(disable=True)


# Funçoes

In [0]:
# Função que deixa colunas minúsculas
def normalize_columns(df):
    df.columns = [unidecode(col).lower() for col in df.columns]
    return df

# Função para normalizar os valores das colunas de texto
def normalize_text_values(df):
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].apply(lambda x: unidecode(x).lower() if isinstance(x, str) else x)
    return df

#Lendo os dados

In [0]:
x_train = pd.read_parquet('data/x_prepared.parquet')

y_train = pd.read_excel('data/Seguro Saúde - Modelagem.xlsx', sheet_name='MODELAGEM')
y_train = normalize_columns(y_train) 
y_train = normalize_text_values(y_train)

y_train = y_train[['matricula', 'valor']]

display(x_train.head())
display(y_train.head())

#Boruta

In [0]:
rf = RandomForestRegressor(n_jobs=-1, random_state=42)

boruta = BorutaPy(
    estimator=rf,
    n_estimators='auto',
    verbose=2,
    random_state=42,
    max_iter=200
)

boruta.fit(x_train.drop('matricula', axis=1).values, y_train.drop('matricula', axis=1).values.ravel())

In [0]:
selected_features = x_train.drop('matricula', axis=1).columns[boruta.support_].tolist()
print("Selecionadas:", selected_features)


In [0]:
feature_ranking = pd.DataFrame({
    'feature': x_train.drop('matricula', axis=1).columns,
    'ranking': boruta.ranking_,
    'selected': boruta.support_
}).sort_values(by='ranking')

display(feature_ranking)


In [0]:
feature_ranking.to_csv('artifacts/boruta_enginneered_features_ranking.csv', index=False)

Concluímos que as variáveis que sao estatiscamente melhores que variáveis de barulho sao:
- num__idade
- cat__fumante_x_regiao_0.0
- num__idade_x_imc
- num__idade_x_filhos
- num__fumante_x_imc 
 
 

#Feature Importance com Random Forest

Queremos confirmar o resultado do boruta vendo o grau de importancia dado pelo random forest

In [0]:
X = x_train.drop('matricula', axis=1)

rf = RandomForestRegressor(random_state=42)
rf.fit(X, y_train.drop('matricula', axis=1))

# Importância das features
importances = rf.feature_importances_
feature_importance = pd.Series(importances, index=x_train.drop('matricula', axis=1).columns).sort_values(ascending=True)

top_features = feature_importance.tail(10)

# Plot
plt.figure(figsize=(10, len(top_features) * 0.35))
top_features.plot(kind='barh')
plt.title("Importância das Features (Random Forest) - TOP 10")
plt.xlabel("Importância", fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=9)
plt.tight_layout()

plt.savefig('plots/rf_feature_importance_top15.png')

plt.show()

In [0]:
aux = pd.DataFrame(
    {'features_selecionadas': [
        'num__idade',
        'cat__fumante_x_regiao_0.0',
        'num__idade_x_imc',
        'num__idade_x_filhos',
        'num__fumante_x_imc',
        'num__idade_div_imc',
        'num__imc'
    ]}
)

aux.to_csv('artifacts/selected_engineered_features.csv', index=False)

Os resultados confirmam o que o Boruta indica, mas dá mais importancia a idade_div_imc e imc a idade_x_filhos.

Essa discrepância ocorre devido as diferenças de estratégias, onde o boruta foca em variáveis estatisticamente relevantes e de maneira mais conservadora, pois compara com as shadows; enquanto que o modelo de regressao random forest tende a valorizar variáveis que particionam grupos grandes. Por isso podemos ter essa discrepância que é considerada normal e saudável na construçao da variáveis.

Podemos incluir essas duas variáveis que o Boruta nao pegou.

Analisemos a multicolinearidade que deve apontar correlaçoes altas entre essas variáveis extras ao Boruta

#Análise de Multicolinearidade

In [0]:
X_selected = x_train[['num__idade', 'num__idade_x_imc', 
                      'num__idade_x_filhos', 'num__fumante_x_imc', 
                      'cat__fumante_x_regiao_0.0', 'num__imc',
                      'num__idade_div_imc']]

corr_matrix = df_corr = X_selected.corr()
mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)

# Plot
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, mask=~mask, annot=True, fmt=".2f", cmap="coolwarm", cbar=True)
plt.title("Correlação - Diagonal Superior")
plt.tight_layout()

plt.savefig('plots/corr_heatmap_selected_enginneered_features.png')

plt.show()

# Criação do DataFrame com os pares com correlação > 0.8
upper_triangle = corr_matrix.where(mask)
high_corr_pairs = (
    upper_triangle.stack()
    .reset_index()
    .rename(columns={"level_0": "variavel_1", "level_1": "variavel_2", 0: "correlacao"})
    .query("abs(correlacao) > 0.8")
    .sort_values(by="correlacao", ascending=False)
)

# Pares altamente correlacionados
display(high_corr_pairs)

high_corr_pairs.to_csv('artifacts/high_corr_pairs.csv', index=False)

Vemos aqui que das 7 variáveis selecionadas (21 pares possíveis) 3 pares têm alta correlaçao:
- idade e idade_x_imc - 0.88
- idade e idade_div_imc - 0.84
- fumante_x_imc e fumante_x_regiao0 - -0.97 (alta correlaçao negativa)

Podemos trabalhar com as 7 variáveis e excluir as que compoem os 3 pares para modelos lineares como regressao linear e ridge regression